# Run "ETL" script

In [1]:
# Open Spark session 
from pyspark.sql import SparkSession, functions as F


# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Data_Explorer")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/10/04 23:34:49 WARN Utils: Your hostname, James-N580VD-DM229T resolves to a loopback address: 127.0.1.1; using 172.22.107.196 instead (on interface eth0)
22/10/04 23:34:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/04 23:34:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Postcode Transaction Analysis
The objective of this section is to verify if a customer's details have been recorded correctly

In [2]:
dataset = spark.read.parquet('../data/curated/full_dataset/')

In [6]:
dataset.columns

['order_id',
 'merchant_abn',
 'user_id',
 'dollar_value',
 'order_datetime',
 'consumer_id',
 'state',
 'postcode',
 'name',
 'address',
 'gender',
 'sa3name',
 'sa4name',
 'electoraterating',
 'SA3_NAME_2016',
 'electorate',
 'Number of individuals lodging an income tax return',
 'Average taxable income or loss',
 'Median taxable income or loss',
 'Proportion with salary or wages',
 'Count salary or wages',
 'Average salary or wages',
 'Median salary or wages',
 'Proportion with net rent',
 'Count net rent',
 'Average net rent',
 'Median net rent',
 'Average total income or loss',
 'Median total income or loss',
 'Average total deductions',
 'Median total deductions',
 'Proportion with total business income',
 'Count total business income',
 'Average total business income',
 'Median total business income',
 'Proportion with total business expenses',
 'Count total business expenses',
 'Average total business expenses',
 'Median total business expenses',
 'Proportion with net tax',
 'C

In [ ]:
# Lets try looking at some of the possible aggregation attributes 'postcode','sa3name', 'sa4name', 'electoraterating', 'SA3_NAME_2016', 'electorate'

In [27]:
from codecs import ascii_encode


dataset.groupBy(['electoraterating','state']).agg(F.avg('dollar_value').alias('dollar_avg'), F.avg('Proportion with salary or wages').alias('AVG_job_pot')).sort('dollar_avg', ascending=False)

electoraterating,state,dollar_avg,AVG_job_pot
Outer Metropolitan,WA,162.10137457495566,1.0
Inner Metropolitan,QLD,161.95596260030928,1.0
Rural,ACT,161.37831281349753,1.0
Outer Metropolitan,TAS,161.1370434353406,1.0
Inner Metropolitan,TAS,161.03532053220445,1.0
Rural,NT,160.41877305512617,1.0
Inner Metropolitan,ACT,160.1090136904166,1.0
Inner Metropolitan,SA,159.87970617166403,1.0
Rural,QLD,159.46443485014788,1.0
Rural,TAS,159.31208605207823,1.0
